# Running InstructLab with a GPU

<ul>
<li>Contributors: InstructLab team and IBM Research Technology Education team:
<li>Questions and support: kochel@us.ibm.com, IBM.Research.JupyterLab@ibm.com
<li>Version: 1.0.13
<li>Release date: 2025-03-20
<li>Compute requirements: Colab with GPU
</ul>

# Running this Notebook

**IMPORTANT:** This notebook must be run within a Colab GPU runtime. You can check you are running with a GPU by selecting Runtime-> Change Runtime Type and confirming that a GPU Runtime is selected. While this notebook can be started on a free Colab account, the GPUs availabe with a free access do not have sufficient memory to run InstructLab training.

You can run this notebook either:
- Running All Cells by selecting Runtime->Run all
- Cell by cell by selecting the arrow on each code cell and running them sequentially.

Once the Configuring Instructlab section has been run, the other sections of this notebook can be repeatedly run on other data sets.

# Section 1. Configure InstructLab

## Step 1.1 Environment Configuration
Replicate the ilab data repository containing the pip requirements and data files and run the pip installs that require a reset.

**IMPORTANT:** Run the next cell, allow it to complete running, then Restart the session , run the following cell to specify parameters and then you can run the remainder of the notebook.

After selecting parameters, the remainder of this notebook can be run either:
- Running All Cells by selecting Runtime->Run cell and below
- Cell by cell by selecting the arrow on each code cell and running them sequentially.


In [ ]:
# Run this cell, then perform the requested Reset
import os
if not os.path.exists("ilab"):
    !git clone https://github.com/KenOcheltree/ilab.git
!pip install numpy==1.26.4 torch==2.5.1 psutil==7.0.0 pillow==10.4.0

## Step 1.2 Optionally, provide your own InstructLab QNA data set

You can optionally provide your own InstructLab QNA file for processing in this step.

**Note:** You may want to run this notebook with an existing dataset before creating your own to understand the InstructLab flow.

Follow these steps to add your own dataset:
1. Create your own qna.yaml file following the directions on the InstructLab taxonomy [readme](https://github.com/instructlab/taxonomy).
1. Create a questions.txt file with related sample questions to use on inferencing.
1. Add your qna.yaml and sample questions.txt files to the /content/ilab/data/your_content_1 folder or the /content/ilab/data/your_content_2 folder by dragging and dropping them in the desired folder.
1. Double click on the /content/ilab/config.json file to edit and specify the qna_location where your data resides within the Dewey Decimal classification system. Close and save the config.json file.
1. You can now specify to run with your own data by selecting **Your Content 1** or **Your Content 2** in the next code cell.

## Step 1.3 Select InstructLab Parameters
Run this next cell, select the following parameters, then follow the direction in the next text cell to run the notebook.

We've provided question-and-answer files for these datasets: "2024 Oscar Awards Ceremony" and "Quantum Roadmap and Patterns" and "Artificial Intelligence Agents". Feel free to choose one of these datasets, or select your own custom dataset in the cell below.

In [ ]:
# Run this second cell to show parameters
import ipywidgets as widgets
#See instructions on placing your hf_token in colab userdata
from google.colab import userdata
hf_token=userdata.get('hf_token')
data_set = widgets.ToggleButtons(
    options=['logs-attacker-identification','logs-pattern-description','logs-service-identification','logs-sus-activity-duration'],
    description='Dataset:', disabled=False, button_style='', style={"button_width": "auto"}
)
sdg_pipe = widgets.ToggleButtons(
    options=['Simple', 'Full with GPU'],
    description='Processing:', disabled=False, button_style='', style={"button_width": "auto"}
)
instr=widgets.ToggleButtons(
    options=['Default (>450)','>15', '>50', '>200', '>500', '>1000'],
    description='# of QNAs:', disabled=False, button_style='', style={"button_width": "auto"}
)
train_pipe = widgets.ToggleButtons(
    options=['Simple with GPU','Accelerated GPU'],
    description='Processing', disabled=False, button_style='', style={"button_width": "auto"}
)
epoch=widgets.ToggleButtons(
    options=['1', '2', '3', '4', '5', '10', '15'],
    description='Epochs:', disabled=False, button_style='', style={"button_width": "auto"}
)
it=widgets.ToggleButtons(
    options=['1', '3', '5','10','20','50','100','200'],
    description='Iterations:', disabled=False, button_style='', style={"button_width": "auto"}
)
questions=widgets.ToggleButtons(
    options=['Yes','No'],
    description='Live Q&A:', disabled=False, button_style='', style={"button_width": "auto"}
)
download=widgets.ToggleButtons(
    options=['Yes','No'],
    description='Download:', disabled=False, button_style='', style={"button_width": "auto"}
)
print("\nSelect the Dataset for this run:")
display(data_set)
print("Select the Synthetic Data Generation parameter to use:")
sdg_pipe.value='Simple'
display(sdg_pipe)
instr.value = 'Default (>450)'
display(instr)
print("Select the Training parameters to use:")
train_pipe.value='Simple with GPU'
#display(train_pipe)
epoch.value="3"
display(epoch)
it.value="5"
display(it)
print("Select what to do with the model after training:")
questions.value="Yes"
display(questions)
download.value="No"
display(download)
print("After selecting the parameters, select the next cell and then choose Runtime->Run cell and below")
print("When that run completes, you can come here, choose different parameters and rerun at the next cell with Runtime->Run cell and below")
print("Note: You can also go back and rerun individual sections of the notebook with different parameters.")


Select the Dataset for this run:


ToggleButtons(description='Dataset:', options=('cybersecurity',), style=ToggleButtonsStyle(button_width='auto'…

Select the Synthetic Data Generation parameter to use:


ToggleButtons(description='Processing:', options=('Simple', 'Full with GPU'), style=ToggleButtonsStyle(button_…

ToggleButtons(description='# of QNAs:', options=('Default (>450)', '>15', '>50', '>200', '>500', '>1000'), sty…

...

ToggleButtons(description='Iterations:', index=2, options=('1', '3', '5', '10', '20', '50', '100', '200'), sty…

Select what to do with the model after training:


ToggleButtons(description='Live Q&A:', options=('Yes', 'No'), style=ToggleButtonsStyle(button_width='auto'), v…

ToggleButtons(description='Download:', index=1, options=('Yes', 'No'), style=ToggleButtonsStyle(button_width='…

After selecting the parameters, select the next cell and then choose Runtime->Run cell and below
When that run completes, you can come here, choose different parameters and rerun at the next cell with Runtime->Run cell and below
Note: You can also go back and rerun individual sections of the notebook with different parameters.


## 1.4 Complete Environment Set Up and Optionally Run All
This next code cell installs the remainder of the reuired pip packages and takes about 7 minutes to run.

If you perform **Runtime->Run cell and below** on this cell, the rest of notebook will take about an hour to run. After running, it will present a prompt for providing questions to the pre-trained and trained models to test improvements in the model.

**Note:** Please ignore the pip dependency errors that appear in the output of the pip installs. They do not affect the successful running of Instructlab.

In [3]:
# Run the rest of the notebook by selecting this third cell and choosing "Runtime->Run cell and below"
!pip cache remove llama_cpp_python
!pip install -r ilab/requirements_gpu.txt
!pip list

Files removed: 0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 32.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 98.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 k

Wrap code cell output for ease of reading

In [4]:
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Step 1.5 Check Starting Configuration
### Check InstructLab Version

Check that InstructLab is installed properly and is configured for using a GPU.

The first line from 'InstructLab' section will give the InstructLab version.

In [5]:
!ilab system info

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA A100-SXM4-40GB, compute capability 8.0, VMM: yes
Platform:
  sys.version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
  sys.platform: linux
  os.name: posix
  platform.release: 6.1.85+
  platform.machine: x86_64
  platform.node: 607bb9a9d368
  platform.python_version: 3.11.12
  os-release.ID: ubuntu
  os-release.VERSION_ID: 22.04
  os-release.PRETTY_NAME: Ubuntu 22.04.4 LTS
  memory.total: 83.48 GB
  memory.available: 81.38 GB
  memory.used: 1.23 GB

InstructLab:
  instructlab.version: 0.24.3
  instructlab-dolomite.version: 0.2.0
  instructlab-eval.version: 0.5.1
  instructlab-quantize.version: 0.1.0
  instructlab-schema.version: 0.4.2
  instructlab-sdg.version: 0.7.3
  instructlab-training.version: 0.7.0

Torch:
  torch.version: 2.5.1+cu124
  torch.backends.cpu.capability: AVX512
  torch.version.cuda: 12.4
  torch.version.hip: None
  tor

<a id="IL1_check"></a>
## Perform Imports and Check for a GPU

This code cell checks for a GPU in the configuration. This notebook requires a GPU in the configuration to run properly.

In [6]:
import os
import torch
from IPython.display import Image, display
from datasets import load_dataset
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
import subprocess
import shutil
import ruamel.yaml
os.environ['NUMEXPR_MAX_THREADS'] = '64'
Norm = "<p style='font-family:IBM Plex Sans;font-size:20px'>"

notebook_dir='/content/ilab/'
os.chdir(notebook_dir)

with open('config.json', 'r') as f:
    jsonData = json.load(f)

## torch and cuda version check
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

if torch.cuda.is_available() is False:
    print("No GPU in configuration")
else:
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    print("GPU(s) are Available")
    gpus=torch.cuda.device_count()
    if gpus==1:
      gpu_type=torch.cuda.get_device_name(0)
      print("One GPU of Type: ", gpu_type)
    else:
      print("ERROR: More than 1 GPU in configuration: ",gpus)
print("Starting directory: "+ os.getcwd())

torch:  2.5 ; cuda:  cu124
GPU(s) are Available
One GPU of Type:  NVIDIA A100-SXM4-40GB
Starting directory: /content/ilab


<a id="IL1_config"></a>
## Step 1.6 Configure InstructLab

### Create InstructLab config file
The InstructLab configuration is captured in the *config.yaml* file. This step creates the config.yaml file and sets:
- **taxomony_path = taxonomy** - the root location of the taxonomy is set to the taxonomy folder in instructlab-latest
- **model_path = models/merlinite-7b-lab-Q4_K_M.gguf** - the default model is set to merlinite

**Note:** The default directories for InstructLab are the following. If you initialize InstructLab on your own system, it will default to the following:
* **Downloaded Models:**  ~/.cache/instructlab/models/ - Contains all downloaded large language models, including the saved output of ones you generate with ilab.
* **Synthetic Data:** ~/.local/share/instructlab/datasets/ - Contains data output from the SDG phase, built on modifications to the taxonomy repository.
* **Taxonomy:** ~/.local/share/instructlab/taxonomy/ - Contains the skill and knowledge data.
* **Training Output:** ~/.local/share/instructlab/checkpoints/ - Contains the output of the training process.
* **config.yaml:** ~/.config/instructlab/config.yaml - Contains the config.yaml file

In [7]:
#Remove Colab Sample directory
if os.path.exists("sample_data"):
    print("removing sample_data")
    shutil.rmtree("sample_data")
    os.chdir("ilab")

#Initialize ilab
base_dir="/root/"
##Choose the base model as granite or mixtral
model_dir="models"
model_name="meta-llama/Llama-3.2-3B-Instruct"
model_path = os.path.join(model_dir, model_name)

taxonomy_path='taxonomy'

## Define the file name
file_name = "config.yaml"
if os.path.exists(file_name):
    os.remove(file_name)
    print(f"ilab was already initialized. {file_name} has been deleted. Reinitialized")
else:
    print(f"ilab was not initialized yet. {file_name} does not exist.")

##Remove old data
if os.path.exists("taxonomy"):
    print("removing taxonomy")
    shutil.rmtree("taxonomy")
if os.path.exists(base_dir+".cache/instructlab"):
    print("removing " + base_dir+".cache/instructlab")
    shutil.rmtree(base_dir+".cache/instructlab")
if os.path.exists(base_dir+".config/instructlab"):
    print("removing " + base_dir+".config/instructlab")
    shutil.rmtree(base_dir+".config/instructlab")
if os.path.exists(base_dir+".local/share/instructlab"):
    print("removing " + base_dir+".local/share/instructlab")
    shutil.rmtree(base_dir+".local/share/instructlab")

print(f"ilab model is {model_path}.")
print('#############################################################')
print(' ')

command = f"""
ilab config init<<EOF
{taxonomy_path}
Y
{model_path}
0
EOF
"""

## Using the ! operator to run the command
!echo "Running ilab config init"
!{command}

ilab was not initialized yet. config.yaml does not exist.
ilab model is models/meta-llama/Llama-3.2-3B-Instruct.
#############################################################
 
Running ilab config init

----------------------------------------------------
         Welcome to the InstructLab CLI
  This guide will help you to setup your environment
----------------------------------------------------

Please provide the following values to initiate the environment [press 'Enter' for default options when prompted]
Path to taxonomy repo [/root/.local/share/instructlab/taxonomy]: `taxonomy` seems to not exist or is empty.
Should I clone https://github.com/instructlab/taxonomy.git for you? [Y/n]: Cloning https://github.com/instructlab/taxonomy.git...
Path to your model [/root/.cache/instructlab/models/granite-7b-lab-Q4_K_M.gguf]: 
Generating config file:
    /root/.config/instructlab/config.yaml

INFO 2025-04-12 09:48:36,353 instructlab.config.init:262: Detecting hardware...
Please choose a 

### Display the config.yaml file
We examine the base configuration for identifying parameters for changing in the next step.

In [8]:
##to copy config.yaml to local directory
!cp /root/.config/instructlab/config.yaml .
!cat config.yaml

# Chat configuration section.
chat:
  # Predefined setting or environment that influences the behavior and responses of
  # the chat assistant. Each context is associated with a specific prompt that
  # guides the assistant on how to respond to user inputs. Available contexts:
  # default, cli_helper.
  # Default: default
  context: default
  # Directory where chat logs are stored.
  # Default: /root/.local/share/instructlab/chatlogs
  logs_dir: /root/.local/share/instructlab/chatlogs
  # The maximum number of tokens that can be generated in the chat completion. Be
  # aware that larger values use more memory.
  # Default: None
  max_tokens:
  # Model to be used for chatting with.
  # Default: /root/.cache/instructlab/models/granite-7b-lab-Q4_K_M.gguf
  model: models/meta-llama/Llama-3.2-3B-Instruct
  # Filepath of a dialog session file.
  # Default: None
  session:
  # Controls the randomness of the model's responses. Lower values make the output
  # more deterministic, while higher v

### Customize LLM Models and copy to notebook for use

This cell changes the models to use for the generate stage. The mistral model as the teacher model in the generate step and as the student model to be trained.

If you want to customize other models for generation or the training phase, you would specify the models in this step.

This step specifies that the models to be used will be from this notebook's models directory.

In [9]:
##Use ruamel.yaml to load the yaml file to preserve comments
yaml = ruamel.yaml.YAML()
with open('config.yaml', 'r') as file:
    config = yaml.load(file)

##Upate to use the same models and just change the directory
teacher_model_path = "models/mistralai/Mistral-7B-Instruct-v0.2"
base_model_path = "models/meta-llama/Llama-3.2-3B-Instruct"
##judge_model_path = "models/prometheus-eval/prometheus-8x7b-v2.0"

##config['evaluate']['mt_bench']['judge_model'] = judge_model_path
##config['evaluate']['mt_bench_branch']['judge_model'] = judge_model_path
config['generate']['model'] = teacher_model_path
config['generate']['teacher']['model_path']= teacher_model_path
##config['train']['phased_mt_bench_judge']=judge_model_path

#Update GPU information
config['evaluate']['gpus']=gpus
config['generate']['teacher']['vllm']['gpus']=gpus
config['serve']['vllm']['gpus']=gpus
config['train']['nproc_per_node']=gpus
config['metadata']['gpu_count']=gpus
if gpus==1:
  config['train']['device']="cuda"
  if gpu_type[:6]=="NVIDIA":
    config['metadata']['gpu_manufacturer']="Nvidia"
    config['metadata']['gpu_family']=gpu_type[7:]

## Save the updated config.yaml file
yaml.default_flow_style=False
with open('config.yaml', 'w') as file:
    yaml.dump(config, file)

##copy the config file to the .config/instructlab/ where it is used by InstructLab
!cp config.yaml {base_dir}.config/instructlab/

print("Updated config.yaml successfully.\n")
!cat config.yaml

Updated config.yaml successfully.

# Chat configuration section.
chat:
  # Predefined setting or environment that influences the behavior and responses of
  # the chat assistant. Each context is associated with a specific prompt that
  # guides the assistant on how to respond to user inputs. Available contexts:
  # default, cli_helper.
  # Default: default
  context: default
  # Directory where chat logs are stored.
  # Default: /root/.local/share/instructlab/chatlogs
  logs_dir: /root/.local/share/instructlab/chatlogs
  # The maximum number of tokens that can be generated in the chat completion. Be
  # aware that larger values use more memory.
  # Default: None
  max_tokens:
  # Model to be used for chatting with.
  # Default: /root/.cache/instructlab/models/granite-7b-lab-Q4_K_M.gguf
  model: models/meta-llama/Llama-3.2-3B-Instruct
  # Filepath of a dialog session file.
  # Default: None
  session:
  # Controls the randomness of the model's responses. Lower values make the output
  #

<a id="IL1_down"></a>
## Step 1.7 Download Models
The models that will be used in the InstructLab processing are downloaded in this step. Additional steps can be added if other models are used in processing.

The merlinite model will be used as the teacher model for the simple pipeline in the **Training with InstructLab** section.

The mistral-7b-instruct-v0.2.Q4_K_M model will be used as the teacher model for the full pipeline in that section.

The granite07b-lab.gguf model is a quantized version of the granite-7b-lab model.

In [10]:
models_dir="models"

!ilab model download --repository meta-llama/Llama-3.2-3B-Instruct --hf-token {hf_token} --model-dir {models_dir}
!ilab model download --repository mistralai/Mistral-7B-Instruct-v0.2 --hf-token {hf_token} --model-dir {models_dir}

INFO 2025-04-12 09:49:00,518 instructlab.model.download:77: Downloading model from Hugging Face:
    Model: meta-llama/Llama-3.2-3B-Instruct@main
    Destination: models
INFO 2025-04-12 09:49:00,835 huggingface_hub.file_download:1652: Downloading '.gitattributes' to 'models/meta-llama/Llama-3.2-3B-Instruct/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'
.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 11.1MB/s]
Download complete. Moving file to models/meta-llama/Llama-3.2-3B-Instruct/.gitattributes
INFO 2025-04-12 09:49:00,897 huggingface_hub.file_download:1684: Download complete. Moving file to models/meta-llama/Llama-3.2-3B-Instruct/.gitattributes
INFO 2025-04-12 09:49:00,950 huggingface_hub.file_download:1652: Downloading 'LICENSE.txt' to 'models/meta-llama/Llama-3.2-3B-Instruct/.cache/huggingface/download/cyBuwAu93UXke23CJCWORBYR70A=.085b47c1575cb889b7024030e60b78f54f0b8c9e.incomplete'
LICENSE.txt: 100% 7.71k/7.71k [00:00<

<a id="IL2_0"></a>
# Section 2. Generating Synthetic Data


This section demonstrates training with InstructLab. This section is part of a sequential notebook. Before running this section of the notebook, please ensure that you have run the Configuring InstructLab section of this notebook.

In this section, we will demonstrate:
- Creating a question and answer data file
- Generating synthetic data for training
- Training the LLM with the generated data

The steps in this section are as follows:
* Step 2.1 Specify the Data for this Run
* Step 2.2 Create the Taxonomy Data Repository
* Step 2.3 Generate Synthetic Data
* Step 2.4 Train Model

<a id="IL2_data"></a>
## Step 2.1 Specify the Data for this Run

We've provided question-and-answer files for these datasets: "2024 Oscar Awards Ceremony", "Quantum Roadmap and Patterns" and "Artificial Intelligence Agents". Feel free to choose one of these datasets, or select your own custom dataset in the cell below.

### Optionally, Create your own data set for InstructLab

You can optionally provide your own InstructLab QNA file for processing in this step.

Follow these steps to add your own dataset:
1. Create your own qna.yaml file following the directions on the InstructLab taxonomy [readme](https://github.com/instructlab/taxonomy).
1. Create a questions.txt file with related sample questions to use on inferencing.
1. Add your qna.yaml and sample questions.txt files to the /content/ilab/data/your_content_1 folder or the /content/ilab/data/your_content_2 folder by dragging and dropping them in the desired folder.
1. Double click on the /content/ilab/config.json file to edit and specify the qna_location where your data resides within the Dewey Decimal classification system. Close and save the config.json file.
1. You can now specify to run with your own data by selecting **Your Content 1** or **Your Content 2** in the next code cell.

In [11]:
print("\nSelect the QNA dataset to add:")
display(data_set)
print("After choosing your dataset, please select and run the following cell")


Select the QNA dataset to add:


ToggleButtons(description='Dataset:', options=('cybersecurity',), style=ToggleButtonsStyle(button_width='auto'…

After choosing your dataset, please select and run the following cell


In [12]:
print("Step 2.2 Choose the Dataset for this Run")
if data_set.value=='cybersecurity':
    use_case="cybersecurity"
else:
    use_case="undefined"

if use_case=="undefined":
    print("ERROR: Undefined data set: " + data_set.value + " data")
else:
    qna_file="data/" + use_case + "/qna.yaml"
    qna_location=jsonData["use_cases"][use_case]["qna_location"]

    print("Using " + data_set.value + " data")

Step 2.2 Choose the Dataset for this Run
Using cybersecurity data


<a id="IL2_taxonomy"></a>
## Step 2.2 Create the Taxonomy Data Repository
Delete the prior repository, clone the empty taxonomy repository and place the QNA file


In [ ]:
#Delete the prior repository and clone the empty taxonomy repository
print("Delete the prior repository and clone the empty taxonomy repository")
shell_command1 = f"rm -rf taxonomy"
taxonomy_repo=jsonData["taxonomy_repo"]
shell_command2 = f"git clone https://github.com/BHnila/taxonomy.git"
!{shell_command1}
!{shell_command2}

print("Verify the taxonomy")
!ilab taxonomy diff

Delete the prior repository and clone the empty taxonomy repository
Cloning into 'taxonomy'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 135 (delta 16), reused 131 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (135/135), 1.75 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Show the QNA file
version: 3
domain: cybersecurity
created_by: BHnila
seed_examples:
  - context: |
      Brute-force attacks are a prevalent phenomenon that is getting harder
      to successfully detect on a network level due to increasing volume and en-
      cryption of network traffic and growing ubiquity of high-speed networks.
      Although the research in this field advanced considerably, there still remain
      classes of attacks that are undetectable.
    questions_and_answers:
      - question: |
          Why is detecting SSH, Telnet, and RDP brute-force attacks more d

<a id="IL2_generate"></a>
## Step 2.3. Set data generation parameters

#### Select pipeline

InstructLab has three primary pipelines that can be used: simple, full and acellerated:
- The **simple pipeline** runs fast and can be used for initial model and data testing.
- The **full pipeline** runs all of the InstrctLab steps and takes more time but produces a better tuned model.

**Note:** If you are running with a new or modifed dataset, you may want to use the **Simple pipeline** for the first run to verify the configuration

#### Sepect number of samples to generate

Data generation takes 19 minutes for generating 15 synthetic data samples. You may wish to generate a small number on your first run to verify the QNA dataset format.

To produce **sufficient synthetic data** to focus training on the new material, **about 30 synthetic questions and answer pairs need to be generated** for each question and answer pair provided. This will require a proportionally longer time to generate, but will provide better training.

Before following these instructions, ensure the existing model you are adding skills or knowledge to is still running. Alternatively, ilab data generate can start a server for you if you provide a fully qualified model path via --model.

To generate a synthetic dataset based on your newly added knowledge or skill set in taxonomy repository, run the following command:

    ilab data generate

#### **Simple Pipeline**

The Simple Pipeline works solely with Merlinite 7b Lab as the teacher model. The Simple Pipeline is called without GPU acceleration as follows:

    ilab data generate --pipeline simple

#### **Full Pipeline**

The Full Pipeline runs the full processing with a GPU. Currently, the Full Pipeline only supports the Mixtral and Mistral Instruct Family models as the teacher model.  This is due to only supporting specific model prompt templates.

Using a non-default model such as Mixtral-8x7B-Instruct-v0.1) to generate data with the Full Pipeline:

    ilab data generate --model ~/.cache/instructlab/models/mistralai/mixtral-8x7b-instruct-v0.1 --pipeline full --gpus 4

**Note** Synthetic Data Generation can take from 2 minutes to 1+ hours to complete, depending on your computing resources.

In [ ]:
print("Select Pipeline to use")
display(sdg_pipe)
display(instr)
print("After making your selections for data generation, please select and run the following cell")

Select Pipeline to use


ToggleButtons(description='Processing:', index=1, options=('Simple', 'Full with GPU'), style=ToggleButtonsStyl…

ToggleButtons(description='# of QNAs:', index=2, options=('Default (>450)', '>15', '>50', '>200', '>500', '>10…

After making your selections for data generation, please select and run the following cell


### 2.4 Run data generation
Data generation with a GPU can take 2 minutes or more to generate 15 synthetic data samples. It takes proportionately longer to generate more samples.

In [ ]:
##to copy config.yaml to local directory
!cp ./config.yaml /root/.config/instructlab/config.yaml

In [ ]:
gen_directory = "data/"+ use_case+"/ilab_generated/"
if instr.value == "Default (>450)":
        sdg_factor=""
elif instr.value == '>15':
    sdg_factor="--sdg-scale-factor 1"
elif instr.value == '>50':
    sdg_factor="--sdg-scale-factor 3"
elif instr.value == '>200':
    sdg_factor="--sdg-scale-factor 13"
elif instr.value == '>500':
    sdg_factor="--sdg-scale-factor 33"
else:
    sdg_factor="--sdg-scale-factor 67"

sdg_pipe.value = 'Full with GPU'

if sdg_pipe.value == 'Full with GPU':
    pipeline = 'full'
    model = ''
#   model = '--model models/instructlab/granite-7b-lab'
    gpus = '--gpus 1'
else:
    print("ERROR: Undefined pipeline")

il_data_path= '/root/.local/share/instructlab/datasets/'
#Remove old data so there is only one test_merlinite and train_merlinite after generation
print("Remove old datasets")
!rm -rf {il_data_path}*
#shell_command = f"ilab --verbose data generate {model} --num-cpus 10 {gpus} {sdg_factor} --taxonomy-path taxonomy --pipeline {pipeline} --max-num-tokens 512"
shell_command = f"ilab data generate --taxonomy-path taxonomy --pipeline full --max-num-tokens 512"

print("Generating data")
print("Running: !"+shell_command)
!{shell_command}

#Rename results to  test_gen.jsonl and train_gen.jsonl and move to local data directory
if not os.path.exists(gen_directory):
    print("Create directory: " + gen_directory)
    !mkdir {gen_directory}
file_cnt=0
try:
    for dirname in os.listdir(il_data_path):
        date_path=il_data_path+'/'+ dirname + '/'
        for filename in os.listdir(date_path):
            if filename[:6]=='train_':
                train_name= 'train_gen.jsonl'
                print('Renaming '+ filename+ ' to ' + train_name)
                !mv {date_path+filename} {gen_directory+train_name}
                file_cnt+=1
            elif filename[:5]=='test_':
                test_name= 'test_gen.jsonl'
                print('Renaming '+ filename+ ' to ' + test_name)
                !mv {date_path+filename} {gen_directory+test_name}
                file_cnt+=1
    if file_cnt < 2:
        print("ERROR: train_gen.jsonl and/or test.jsonl not created")
    elif os.path.getsize(gen_directory+train_name) == 0:
        print("ERROR: train_gen.jsonl file is empty")
    elif os.path.getsize(gen_directory+test_name) == 0:
        print("ERROR: test_gen.jsonl file is empty")
    else:
        print("Training and test files successfully created in: " + gen_directory)
except:
    print("Error running ilab generate, no synthetic data generated")

Remove old datasets
Generating data
Running: !ilab data generate --taxonomy-path taxonomy --pipeline full --max-num-tokens 512
INFO 2025-04-06 20:17:31,019 instructlab.process.process:297: Started subprocess with PID 14836. Logs are being written to /root/.local/share/instructlab/logs/generation/generation-2b08293e-1324-11f0-ba2d-0242ac1c000c.log.
INFO 2025-04-06 20:17:33,206 instructlab.model.backends.vllm:115: Trying to connect to model server at http://127.0.0.1:8000/v1
INFO 2025-04-06 20:17:34,556 instructlab.model.backends.vllm:332: vLLM starting up on pid 14871 at http://127.0.0.1:34523/v1
INFO 2025-04-06 20:17:34,556 instructlab.model.backends.vllm:123: Starting a temporary vLLM server at http://127.0.0.1:34523/v1
INFO 2025-04-06 20:17:34,556 instructlab.model.backends.vllm:138: Waiting for the vLLM server to start at http://127.0.0.1:34523/v1, this might take a moment... Attempt: 1/120
INFO 2025-04-06 20:17:37,951 instructlab.model.backends.vllm:138: Waiting for the vLLM server

In [ ]:
# prompt: I need to move all generated data in previous code block to my mounted google drive

import shutil
import os

# Define source and destination directories
source_dir = "data/cybersecurity/ilab_generated/"  # Example source directory
destination_dir = "/content/drive/MyDrive/ilab_data"  # Your desired Google Drive destination

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Iterate through files and directories in source directory
for item in os.listdir(source_dir):
    source_item_path = os.path.join(source_dir, item)
    destination_item_path = os.path.join(destination_dir, item)

    # Check if it's a file
    if os.path.isfile(source_item_path):
        print(f"Copying file: {source_item_path} to {destination_item_path}")
        shutil.copy2(source_item_path, destination_item_path)  # copy2 preserves metadata

    # Check if it's a directory (recursively copy subdirectories)
    elif os.path.isdir(source_item_path):
        print(f"Copying directory: {source_item_path} to {destination_item_path}")
        shutil.copytree(source_item_path, destination_item_path, dirs_exist_ok=True)

print(f"Data successfully copied to {destination_dir}")


Copying file: data/cybersecurity/ilab_generated/test_gen.jsonl to /content/drive/MyDrive/ilab_data/test_gen.jsonl
Copying file: data/cybersecurity/ilab_generated/train_gen.jsonl to /content/drive/MyDrive/ilab_data/train_gen.jsonl
Data successfully copied to /content/drive/MyDrive/ilab_data


### 2.5 Show examples of generated data

In [16]:
print("2.4.3 Show examples of generated data")
gen_directory = "data/"+ use_case+"/ilab_generated/"
for filename in os.listdir(gen_directory):
    if filename[:9]=='train_gen':
        with open(gen_directory+filename, 'r') as syn_file:
            cnt=0
            for line_number, line in enumerate(syn_file):
                if cnt >= 8:
                    break
                jsonLine= json.loads(line)
                syn_user=jsonLine["user"]
                syn_assist=jsonLine["assistant"]
                #Remove "Answer:" and "Response:" from answers for displaying
                if syn_user[:10]=="Question: ":
                    syn_user=syn_user[10:]
                if syn_assist[:8]=="Answer: ":
                    syn_assist=syn_assist[8:]
                cnt+=1
                print("\nQuestion: "+syn_user+"\nAnswer: "+syn_assist)

2.4.3 Show examples of generated data


KeyError: 'user'

## Section 3. Training with InstructLab

### 3.1 Select the model training pipeline

InstructLab has three primary model training pipelines: simple, full (default), and accelerated. For all of the models, the training time can be limited by adjusting the num_epoch paramater. The maximum number of epochs for running the InstructLab end-to-end workflow is 10.

#### **Simple pipeline**

The simple pipeline uses an SFT Trainer on Linux and MLX on MacOS. This type of training takes roughly an hour and produces the lowest fidelity model but should indicate if your data is being picked up by the training process. The simple pipeline only works with Merlinite 7b Lab as the teacher model. For this Linux system, the trained model is saved in the models directory as ggml-model-f16.gguf.

The command form is:

    ilab model train --pipeline simple

**Note:** This process will take a little while to complete (time can vary based on hardware and output of ilab data generate but on the order of 5 to 15 minutes)

#### **Accelerated pipeline**

The accelerated uses the instructlab-training library which supports GPU accelerated and distributed training. The full loop and data processing functions are either pulled directly from or based off of the work in this library. For the accelerated pipeline, the models are saved in the ~/.local/share/instructlab/checkpoints directory. The instructlab command "ilab model evaluate" can be used to choose the best one. Training is support for GPU acceleration with Nvidia CUDA or AMD ROCm. Please see the GPU acceleration documentation for more details. At present, hardware acceleration requires a data center GPU or high-end consumer GPU with at least 18 GB free memory.

The command form is:

    ilab model train --pipeline accelerated --device cuda --data-path <path-to-sdg-data>

In [14]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()
print("Select to Continue or to Train the model")
display(train_pipe)
display(epoch)
display(it)
print("After choosing your training options, please select and run the following cell")

Select to Continue or to Train the model


ToggleButtons(description='Processing', options=('Simple with GPU', 'Accelerated GPU'), style=ToggleButtonsSty…

ToggleButtons(description='Epochs:', index=5, options=('1', '2', '3', '4', '5', '10', '15'), style=ToggleButto…

ToggleButtons(description='Iterations:', index=4, options=('1', '3', '5', '10', '20', '50', '100', '200'), sty…

After choosing your training options, please select and run the following cell


### 3.2 Run the model training

Model training can take 30 minutes or more for 1 epoch and 1 iteration and takes 1 hour for the default paramter values. This minimal training could be used for testing the generation and training for a new set of data.

To produce a higher quality model, more epochs and iterations are needed for refining the model. This will require a proportionally longer time to train the model.

In [ ]:
##to copy config.yaml to local directory
!cp ./config.yaml /root/.config/instructlab/config.yaml

In [17]:
data_path="data/"+ use_case+"/ilab_generated/"
train_data=data_path+"train_gen.jsonl"
model_path="models/meta-llama/Llama-3.2-3B-Instruct"
##model_path='/root/.cache/instructlab/models/instructlab/granite-7b-lab'
trained_model_path="data/"+ use_case+"/new_model/"
train_name= 'train_gen.jsonl'
test_name= 'test_gen.jsonl'

##'Simple (Fast)', 'Accelerated GPU'
file_cnt=0
for filename in os.listdir(data_path):
    if filename[:15]=='train_gen.jsonl': file_cnt+=1
    elif filename[:14]=='test_gen.jsonl': file_cnt+=1
if file_cnt < 2 or os.path.getsize(gen_directory+train_name) < 5 or os.path.getsize(gen_directory+test_name) < 5:
    print("ERROR: train_gen.jsonl and/or test.jsonl are not present or too small")

if not os.path.exists(trained_model_path):
    print("Create directory: " + trained_model_path)
    !mkdir {trained_model_path}
ep=int(epoch.value)
its=int(it.value)
train_pipe.value='Accelerated GPU'
if train_pipe.value=='Simple with GPU':
    print("Train with simple pipeline with a GPU")
    shell_command = f"ilab model train --pipeline simple --device cuda --model-path {model_path} --data-path {data_path} --num-epochs {ep} --iters {its}"
elif train_pipe.value=='Accelerated GPU':
    print("Train accelerated with a GPU")
    #shell_command = f"ilab model train --pipeline accelerated --device cuda --model-path {model_path} --data-path {train_data} --num-epochs {ep} --iters {its}"
    shell_command = f"ilab -v -v model train --pipeline accelerated --device cuda --model-path {'/content/ilab/'+model_path} --data-path {'/content/ilab/'+train_data} --num-epochs 10 --iters 20 --disable-accelerate-full-state-at-epoch"

print("Running: !"+shell_command)
!{shell_command}
# if train_pipe.value=='Accelerated GPU':
#     print("Run ilab model evaluate")
#     !ilab model evaluate --benchmark mmlu
#Move the model to the use_case/new_model directory
print("Moving the trained model to the directory: "+trained_model_path)
!mv /root/.local/share/instructlab/checkpoints/ggml-model-f16.gguf {trained_model_path}

Streaming output truncated to the last 5000 lines.

Epoch 7:  44%|████▍     | 46/104 [00:42<00:53,  1.09it/s]{
    "epoch": 7,
    "step": 774,
    "rank": 0,
    "overall_throughput": 37.303291566761985,
    "lr": 3.355993196827075e-06,
    "cuda_mem_allocated": 19.11925220489502,
    "cuda_malloc_retries": 0,
    "num_loss_counted_tokens": 1774,
    "batch_size": 33,
    "total_loss": 0.04255918827508456,
    "samples_seen": 24256,
    "gradnorm": 5.78125,
    "total_samples": 3257,
    "timestamp": "2025-04-12T10:11:28.370172"
}
 total length: 4782 num samples 30 - rank: 0 num_loss_counted_tokens: 1812
Epoch: 7, Step: 775, Rank: 0, loss = 0.064453125

Epoch 7:  45%|████▌     | 47/104 [00:43<00:51,  1.11it/s]{
    "epoch": 7,
    "step": 775,
    "rank": 0,
    "overall_throughput": 39.31844761274511,
    "lr": 3.355993196827075e-06,
    "cuda_mem_allocated": 25.091779232025146,
    "cuda_malloc_retries": 0,
    "num_loss_counted_tokens": 1687,
    "batch_size": 30,
    "total_loss":

In [19]:
# prompt: I need a code that copies the folder /root/.local/share/instructlab/checkpoints to my Google Drive

import shutil
import os

# Define source and destination directories
source_dir = "/root/.local/share/instructlab/checkpoints"
destination_dir = "/content/drive/MyDrive/instructlab_checkpoints"  # Or any other path in your Google Drive

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy the folder and its contents
try:
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print(f"Successfully copied '{source_dir}' to '{destination_dir}'")
except OSError as e:
    print(f"Error copying folder: {e}")


Successfully copied '/root/.local/share/instructlab/checkpoints' to '/content/drive/MyDrive/instructlab_checkpoints'


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Section 4. Inferencing with the Model

You have now completed InstructLab training. You can run this section to ask questions to both the base and InstructLab trained models and to compare answers.

This third notebook section showcases the generation of synthetic data utilizing InstructLab. It subsequently demonstrates how a large language model (LLM) can be effectively trained on this synthetic dataset. In current notebook, Both the pre-trained LLM and the LLM trained on the generated synthetic data are evaluated against a predefined set of questions to assess their respective performance.

<a id="IL3_3"></a>
## Run Interactive Q&A Session with Base and Trained Models to Evaluate Performance

Run both base and trained models to compare results with interactive questions and and answers.

In [ ]:
print("Do you want to run interactive question on the base and trained models?")
display(questions)
print("After making your choice, please select and run the following cell")

The following are sample questions derived from the data used to generate synthetic data, which was then employed to train the language model.

In [ ]:
# Define a Function to Perform Inference on Base and Trained Models
def model_inference(base_model_path, trained_model_path):
    _DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

    Current conversation:
    Human: {input}
    AI:"""
    base_llm = LlamaCpp(model_path=base_model_path,
                   verbose=False,
                   n_gpu_layers=25,
                   max_tokens=90,
                   temperature=0,
                   top_k=1
                  )
    trained_llm = LlamaCpp(model_path=trained_model_path,
                   verbose=False,
                   n_gpu_layers=25,
                   max_tokens=90,
                   temperature=0,
                   top_k=1
                  )
    PROMPT = PromptTemplate( input_variables=["input"],
                            template=_DEFAULT_TEMPLATE
                            )
    chain1 = PROMPT | base_llm | StrOutputParser()
    chain2 = PROMPT | trained_llm | StrOutputParser()
    while True:
        question = input("Ask me a question (type 'exit' to end): ")
        if question.lower() == 'exit':
            print("Exiting this Q&A session.")
            break
        else:
            print("You asked: ", question)
            answer1 = chain1.invoke(question)
            answer1= answer1.split('Human',1)[0]
            print ("Base Model Answer: ",answer1)
            answer2 = chain2.invoke(question)
            answer2= answer2.split('Human',1)[0]
            print ("Trained Model Answer: ",answer2)

##Display Sample Questions
base_model = notebook_dir +"/models/granite-7b-lab-Q4_K_M.gguf"
trained_model = trained_model_path + "ggml-model-f16.gguf"
if questions.value=='Yes':
  with open(notebook_dir+'/data/' + use_case + '/questions.txt') as f:
      for line in f.readlines():
          display(widgets.HTML(Norm+line))
  print("Processing may take several minutes on the first run...")
  model_inference(base_model, trained_model)

# Section 5. Download the Trained Model
 Now that we have a model trained on our dataset, we can download the trained model for futher testing and use.

In [ ]:
print("Do you want to download the trained model to your local machine?")
display(download)
print("After making your selection, please select and run the following cell")

Select and run the next cell to download if selected.

In [ ]:
from google.colab import files
if download.value=='Yes':
  files.download(trained_model)

<a id="IL3_conclusion"></a>
# Conclusion

This notebook demonstrated utilizing InstructLab for introducing datasets, data generation, model training, and model creation. This notebook produced an InstructLab trained model that was available for inferecing and downloading.

<a id="IL3_learn"></a>
# Learn More

InstructLab uses a novel synthetic data-based alignment tuning method for Large Language Models introduced in this [paper](https://arxiv.org/abs/2403.01081).

This notebook is based on the InstructLab CLI repository available [here](https://github.com/instructlab/instructlab).

Contact us by email to ask questions, discuss potential use cases, or schedule a technical deep dive. The contact email is IBM.Research.JupyterLab@ibm.com.

© 2025 IBM Corporation